## References

**Foundational MOST papers:**
- Monin & Obukhov (1954): Dimensionless characteristics of turbulence in the ground layer
- Businger et al. (1971): Flux-profile relationships in the atmospheric surface layer. *J. Atmos. Sci.*, 28, 181–189
- Högström (1988): Non-dimensional wind and temperature profiles. *Boundary-Layer Meteor.*, 44, 25–60
- Cheng & Brutsaert (2005): Flux-profile relationships for wind and temperature. *J. Atmos. Sci.*, 62, 2112–2132

**Key concepts:**
- ζ = z/L (dimensionless height, where L is Obukhov length)
- Unstable: ζ < 0 (convection enhances mixing)
- Neutral: ζ ≈ 0 (mechanical mixing only)
- Stable: ζ > 0 (stratification suppresses mixing)

## 1. Import Libraries and Define Constants

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import integrate

# Display settings
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: f'{x:.6f}')

print("Libraries imported successfully!")

## 2. Profile Parameters and Constants

Define canonical MOST profile coefficients from literature.

In [ ]:
"""
MOST Profile Parameters
=======================
Each profile defines coefficients for stability functions:
- Unstable (ζ < 0): φ = (1 - a*ζ)^(-b)
- Stable (ζ > 0): φ = 1 + b*ζ [+ c*ζ² for CB05]
"""

profiles = {
    'BD71': {
        'name': 'Businger-Dyer 1971',
        'unstable': {'am': 16, 'ah': 16, 'bm': 0.25, 'bh': 0.5},
        'stable': {'bm': 5, 'bh': 5}
    },
    'HOG88': {
        'name': 'Högström 1988',
        'unstable': {'am': 19.3, 'ah': 11.6, 'bm': 0.25, 'bh': 0.5},
        'stable': {'bm': 6, 'bh': 7.8}
    },
    'CB05': {
        'name': 'Cheng-Brutsaert 2005',
        'unstable': {'am': 16, 'ah': 16, 'bm': 0.25, 'bh': 0.5},
        'stable': {'bm': 6.1, 'bh': 6.1, 'cm': 5.3, 'ch': 5.3}
    }
}

# Numerical integration and convergence parameters
NUM = {
    'PSI_STEPS': 100,        # Integration steps for ψ
    'SINGULAR': 1e-10,       # Singularity tolerance
    'DZ': 1e-8,              # Numerical derivative step
    'NEWTON_TOL': 1e-10,     # Newton's method tolerance
    'NEWTON_MAX_ITER': 100   # Max Newton iterations
}

# Surface parameters
SURFACE = {
    'Z_OVER_Z0': 1000.0  # z/z0 ratio (e.g., z=10m, z0=0.01m)
}

print("Profile parameters defined:")
for prof_key, prof_data in profiles.items():
    print(f"  {prof_key}: {prof_data['name']}")
print(f"\nSurface parameters: z/z0 = {SURFACE['Z_OVER_Z0']}")

## 3. Stability Functions: φ_m and φ_h

These functions describe how stability affects momentum and heat fluxes.

In [ ]:
def phi_m(zeta, prof):
    """
    Momentum stability function φ_m(ζ).
    
    Describes how stability affects momentum flux in the surface layer.
    - Unstable (ζ < 0): φ_m decreases → enhanced mixing
    - Stable (ζ > 0): φ_m increases → suppressed mixing
    
    Parameters:
        zeta (float): Dimensionless height z/L
        prof (str): Profile name ('BD71', 'HOG88', 'CB05')
    
    Returns:
        float: Stability function φ_m
    """
    p = profiles[prof]
    
    if zeta < 0:
        # Unstable: (1 - a_m * ζ)^(-b_m)
        return (1 - p['unstable']['am'] * zeta) ** (-p['unstable']['bm'])
    else:
        # Stable: linear or quadratic
        if prof == 'CB05':
            # CB05 includes quadratic term
            return 1 + p['stable']['bm'] * zeta + p['stable'].get('cm', 0) * zeta**2
        else:
            # BD71, HOG88: linear only
            return 1 + p['stable']['bm'] * zeta

def phi_h(zeta, prof):
    """
    Heat stability function φ_h(ζ).
    
    Analogous to φ_m but for temperature gradients.
    Typically differs from φ_m in unstable regime (different exponent).
    
    Parameters:
        zeta (float): Dimensionless height z/L
        prof (str): Profile name
    
    Returns:
        float: Stability function φ_h
    """
    p = profiles[prof]
    
    if zeta < 0:
        # Unstable: (1 - a_h * ζ)^(-b_h)
        return (1 - p['unstable']['ah'] * zeta) ** (-p['unstable']['bh'])
    else:
        # Stable
        if prof == 'CB05':
            return 1 + p['stable']['bh'] * zeta + p['stable'].get('ch', 0) * zeta**2
        else:
            return 1 + p['stable']['bh'] * zeta

# Test the functions
zeta_test = np.array([-1, -0.1, 0, 0.1, 1])
print("Test: φ_m(ζ) and φ_h(ζ) for BD71 profile:")
print("ζ\t\tφ_m\t\tφ_h")
for z in zeta_test:
    pm = phi_m(z, 'BD71')
    ph = phi_h(z, 'BD71')
    print(f"{z:+.1f}\t\t{pm:.4f}\t\t{ph:.4f}")

## 4. Integral Stability Functions: ψ_m and ψ_h

These are integrals of stability functions used in the log-wind profile.

**Unstable:** Analytical solutions available
**Stable:** Numerical integration required (trapezoidal rule)

In [ ]:
def psi_m(zeta, prof):
    """
    Integral momentum stability function ψ_m(ζ).
    
    Defined as: ψ_m = ∫₀^ζ (1/φ_m - 1) dz'
    
    Used in logarithmic wind profile:
    u(z) = (u*/κ) * [ln(z/z₀) - ψ_m(ζ)]
    
    Unstable: Analytical solution
    Stable: Numerical integration
    
    Parameters:
        zeta (float): Dimensionless height
        prof (str): Profile name
    
    Returns:
        float: Integral stability function ψ_m
    """
    if abs(zeta) < NUM['SINGULAR']:
        return 0.0  # Neutral condition
    
    p = profiles[prof]
    
    if zeta < 0:
        # Unstable: Analytical solution
        # x = (1 - a_m * ζ)^0.25
        # ψ_m = 2*ln((1+x)/2) + ln((1+x²)/2) - 2*atan(x) + π/2
        x = (1 - p['unstable']['am'] * zeta) ** 0.25
        return (
            2 * np.log((1 + x) / 2) +
            np.log((1 + x**2) / 2) -
            2 * np.arctan(x) +
            np.pi / 2
        )
    else:
        # Stable: Numerical integration (trapezoidal rule)
        steps = NUM['PSI_STEPS']
        dz = zeta / steps
        z_vals = np.linspace(dz, zeta, steps)
        integrand = 1 / np.array([phi_m(zi, prof) for zi in z_vals]) - 1
        return np.trapz(integrand, dx=dz)

def psi_h(zeta, prof):
    """
    Integral heat stability function ψ_h(ζ).
    
    Analogous to ψ_m for temperature gradients.
    
    Used in logarithmic temperature profile:
    θ(z) - θ(z₀) = (θ*/κ) * [ln(z/z₀) - ψ_h(ζ)]
    
    Parameters:
        zeta (float): Dimensionless height
        prof (str): Profile name
    
    Returns:
        float: Integral stability function ψ_h
    """
    if abs(zeta) < NUM['SINGULAR']:
        return 0.0
    
    p = profiles[prof]
    
    if zeta < 0:
        # Unstable: Analytical solution
        # y = (1 - a_h * ζ)^0.5
        # ψ_h = 2*ln((1+y)/2)
        y = (1 - p['unstable']['ah'] * zeta) ** 0.5
        return 2 * np.log((1 + y) / 2)
    else:
        # Stable: Numerical integration
        steps = NUM['PSI_STEPS']
        dz = zeta / steps
        z_vals = np.linspace(dz, zeta, steps)
        integrand = 1 / np.array([phi_h(zi, prof) for zi in z_vals]) - 1
        return np.trapz(integrand, dx=dz)

# Test
print("Test: ψ_m(ζ) and ψ_h(ζ) for BD71 profile:")
print("ζ\t\tψ_m\t\tψ_h")
for z in zeta_test:
    psim = psi_m(z, 'BD71')
    psih = psi_h(z, 'BD71')
    print(f"{z:+.1f}\t\t{psim:+.4f}\t\t{psih:+.4f}")

## 5. Richardson Numbers: Ri_g and Ri_b

**Ri_g (Gradient Richardson):** Local stability measure
- Compares static stability to wind shear
- Ri_g < 0: Unstable (convection)
- Ri_g > 0.25: Turbulence suppressed (critical value)

**Ri_b (Bulk Richardson):** Integrated stability measure
- Uses constant ln(z/z₀) term
- More robust for layer-average analysis

In [ ]:
def ri_g(zeta, prof):
    """
    Gradient Richardson number Ri_g(ζ).
    
    Local stability measure:
    Ri_g = (g/θ₀) * (dθ/dz) / (du/dz)²
    
    In MOST scaling:
    Ri_g ≈ ζ * φ_h / φ_m²
    
    Physical interpretation:
    - Ri_g < 0: Unstable (convection dominates)
    - Ri_g ≈ 0: Neutral
    - Ri_g > 0.25: Critical (turbulence suppressed)
    
    Parameters:
        zeta (float): Dimensionless height
        prof (str): Profile name
    
    Returns:
        float: Gradient Richardson number
    """
    pm = phi_m(zeta, prof)
    ph = phi_h(zeta, prof)
    return zeta * ph / (pm ** 2)

def ri_b(zeta, prof):
    """
    Bulk Richardson number Ri_b(ζ).
    
    Integrated stability measure:
    Ri_b ≈ ζ * [ln(z/z₀) - ψ_h] / [ln(z/z₀) - ψ_m]²
    
    Uses constant ln(z/z₀) based on SURFACE['Z_OVER_Z0'].
    
    Physical interpretation:
    - More integrative than local Ri_g
    - Used to detect turbulence decoupling in stable layers
    - Critical value Ri_b,crit ≈ 0.2–0.3
    
    Parameters:
        zeta (float): Dimensionless height
        prof (str): Profile name
    
    Returns:
        float: Bulk Richardson number
    """
    if abs(zeta) < NUM['SINGULAR']:
        return 0.0
    
    ln_z_z0 = np.log(SURFACE['Z_OVER_Z0'])
    if ln_z_z0 <= 0:
        return 0.0  # Invalid z/z0
    
    psim = psi_m(zeta, prof)
    psih = psi_h(zeta, prof)
    
    denominator = (ln_z_z0 - psim) ** 2
    if denominator < 1e-15:
        return 0.0
    
    return zeta * (ln_z_z0 - psih) / denominator

# Test
print("Test: Ri_g(ζ) and Ri_b(ζ) for BD71 profile:")
print("ζ\t\tRi_g\t\tRi_b")
for z in zeta_test:
    rig = ri_g(z, 'BD71')
    rib = ri_b(z, 'BD71')
    print(f"{z:+.1f}\t\t{rig:+.6f}\t\t{rib:+.6f}")

## 6. Generate Reference Data Tables

Create comprehensive tables for all three profiles.

In [ ]:
def generate_reference_table(profile, regime='full'):
    """
    Generate reference data table for a given profile and regime.
    
    Parameters:
        profile (str): Profile name ('BD71', 'HOG88', 'CB05')
        regime (str): 'unstable', 'stable', or 'full'
    
    Returns:
        pd.DataFrame: Reference data with columns [zeta, phi_m, phi_h, Ri_g, Ri_b]
    """
    # Select ζ range
    if regime == 'unstable':
        zeta_range = np.linspace(-10, 0, 101)
    elif regime == 'stable':
        zeta_range = np.linspace(0.001, 10, 100)  # Avoid singularity at 0
    else:  # full
        zeta_range = np.linspace(-10, 10, 201)
    
    data = []
    for z in zeta_range:
        pm = phi_m(z, profile)
        ph = phi_h(z, profile)
        rig = ri_g(z, profile)
        rib = ri_b(z, profile) if abs(z) > 1e-5 else 0.0
        
        data.append({
            'zeta': round(z, 4),
            'phi_m': round(pm, 6),
            'phi_h': round(ph, 6),
            'Ri_g': round(rig, 8),
            'Ri_b': round(rib, 8)
        })
    
    return pd.DataFrame(data)

# Generate tables for all profiles
print("Generating reference tables...\n")
tables = {}
for prof in profiles.keys():
    tables[prof] = generate_reference_table(prof, regime='full')
    print(f"{prof} ({profiles[prof]['name']}):")
    print(tables[prof].head(10))
    print(f"  Shape: {tables[prof].shape}")
    print()

## 7. Visualization: Stability Relationships

In [ ]:
# Create comprehensive plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('MOST Stability Relationships (All Profiles)', fontsize=16, fontweight='bold')

prof_list = list(profiles.keys())
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

# Plot 1: ζ vs Ri_g
ax = axes[0, 0]
for prof, color in zip(prof_list, colors):
    ax.plot(tables[prof]['zeta'], tables[prof]['Ri_g'], 
            label=prof, color=color, linewidth=2)
ax.axhline(0, color='k', linestyle='--', alpha=0.3)
ax.axvline(0, color='k', linestyle='--', alpha=0.3)
ax.set_xlabel('ζ = z/L', fontsize=11)
ax.set_ylabel('Ri_g', fontsize=11)
ax.set_title('Gradient Richardson Number', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: ζ vs Ri_b
ax = axes[0, 1]
for prof, color in zip(prof_list, colors):
    ax.plot(tables[prof]['zeta'], tables[prof]['Ri_b'], 
            label=prof, color=color, linewidth=2)
ax.axhline(0, color='k', linestyle='--', alpha=0.3)
ax.axvline(0, color='k', linestyle='--', alpha=0.3)
ax.set_xlabel('ζ = z/L', fontsize=11)
ax.set_ylabel('Ri_b', fontsize=11)
ax.set_title('Bulk Richardson Number', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: ζ vs φ_m
ax = axes[1, 0]
for prof, color in zip(prof_list, colors):
    ax.plot(tables[prof]['zeta'], tables[prof]['phi_m'], 
            label=prof, color=color, linewidth=2)
ax.set_xlabel('ζ = z/L', fontsize=11)
ax.set_ylabel('φ_m', fontsize=11)
ax.set_title('Momentum Stability Function', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: ζ vs φ_h
ax = axes[1, 1]
for prof, color in zip(prof_list, colors):
    ax.plot(tables[prof]['zeta'], tables[prof]['phi_h'], 
            label=prof, color=color, linewidth=2)
ax.set_xlabel('ζ = z/L', fontsize=11)
ax.set_ylabel('φ_h', fontsize=11)
ax.set_title('Heat Stability Function', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/MOST_stability_relationships.png', dpi=150, bbox_inches='tight')
plt.show()

print("Plot saved to ../data/MOST_stability_relationships.png")

## 8. Statistical Summary of Stability Parameters

In [ ]:
print("Statistical Summary of Reference Data\n")
print("="*70)

for prof in prof_list:
    print(f"\n{prof} ({profiles[prof]['name']}):")
    print("-" * 70)
    print(tables[prof][['Ri_g', 'Ri_b', 'phi_m', 'phi_h']].describe())

## 9. Export CSV Tables

Save reference tables to CSV format for external use.

In [ ]:
import os

# Create data directory if it doesn't exist
os.makedirs('../data', exist_ok=True)

# Export tables for each profile
for prof in prof_list:
    filename = f'../data/MOST_reference_{prof}_full.csv'
    tables[prof].to_csv(filename, index=False)
    print(f"Exported: {filename}")

# Export combined table
combined = pd.concat([
    tables[prof].assign(profile=prof)
    for prof in prof_list
], ignore_index=True)

combined.to_csv('../data/MOST_reference_all_profiles.csv', index=False)
print(f"\nExported: ../data/MOST_reference_all_profiles.csv")

print(f"\nCombined table shape: {combined.shape}")
print(combined.head(10))

## 10. Usage Examples: Parameter Conversions

**Example 1:** Given Ri_g, estimate ζ
**Example 2:** Given ζ, compute all parameters
**Example 3:** Identify critical stability thresholds

In [ ]:
"""
USAGE EXAMPLES: Parameter Conversions
======================================
These examples demonstrate how to use the reference tables for practical stability analysis.
"""

print("EXAMPLE 1: Determine stability regime from Ri_g")
print("-" * 70)

test_rig = [0, 0.05, 0.1, 0.25, 0.5]
prof_ex = 'BD71'

for rig_val in test_rig:
    # Find closest Ri_g value in table
    idx = (tables[prof_ex]['Ri_g'] - rig_val).abs().idxmin()
    row = tables[prof_ex].iloc[idx]
    
    # Classify stability
    if row['Ri_g'] < -0.01:
        regime_name = 'Unstable (Convective)'
    elif -0.01 <= row['Ri_g'] <= 0.01:
        regime_name = 'Neutral'
    elif 0.01 < row['Ri_g'] < 0.25:
        regime_name = 'Stable (Weak)'
    elif row['Ri_g'] >= 0.25:
        regime_name = 'Stable (Strong, Decoupling)'
    else:
        regime_name = 'Unknown'
    
    print(f"Ri_g ≈ {rig_val:+.2f} → ζ = {row['zeta']:+.4f}, Regime: {regime_name}")

print("\n" + "="*70)
print("EXAMPLE 2: Compute stability parameters from ζ")
print("-" * 70)

test_zeta = [-2, -0.5, -0.1, 0, 0.1, 0.5, 2]
for z_val in test_zeta:
    idx = (tables[prof_ex]['zeta'] - z_val).abs().idxmin()
    row = tables[prof_ex].iloc[idx]
    
    print(f"ζ = {row['zeta']:+.3f}: φ_m={row['phi_m']:.4f}, φ_h={row['phi_h']:.4f}, "
          f"Ri_g={row['Ri_g']:+.6f}, Ri_b={row['Ri_b']:+.6f}")

## 11. Critical Values and Thresholds

Identify physically important stability thresholds across all profiles.

In [ ]:
print("CRITICAL STABILITY THRESHOLDS")
print("="*70)

for prof in prof_list:
    print(f"\n{prof} ({profiles[prof]['name']}):")
    print("-" * 70)
    
    df = tables[prof]
    
    # Critical Ri_g (turbulence suppression at ~0.25)
    crit_rig_idx = (df['Ri_g'] - 0.25).abs().idxmin()
    crit_rig_row = df.iloc[crit_rig_idx]
    print(f"  Ri_g ≈ 0.25 (critical): ζ = {crit_rig_row['zeta']:+.4f}")
    
    # Maximum Ri_b in stable
    stable_df = df[df['zeta'] > 0]
    if len(stable_df) > 0:
        max_rib_idx = stable_df['Ri_b'].idxmax()
        max_rib_row = df.iloc[max_rib_idx]
        print(f"  Max Ri_b (stable): ζ = {max_rib_row['zeta']:+.4f}, Ri_b = {max_rib_row['Ri_b']:.6f}")
    
    # Neutral condition (ζ ≈ 0)
    neutral_idx = df['zeta'].abs().idxmin()
    neutral_row = df.iloc[neutral_idx]
    print(f"  Neutral (ζ ≈ 0): φ_m = {neutral_row['phi_m']:.4f}, φ_h = {neutral_row['phi_h']:.4f}")

## Summary

This notebook has demonstrated:

1. **Physics Implementation** — Canonical MOST stability functions (φ_m, φ_h, ψ_m, ψ_h)
2. **Reference Tables** — High-precision (10⁻⁸) calculations across three profiles
3. **Richardson Numbers** — Both gradient (Ri_g) and bulk (Ri_b) measures
4. **Visualization** — Stability relationships and critical thresholds
5. **Data Export** — CSV tables for external use in models and analysis

### Key Findings

- **BD71 vs HOG88:** BD71 is more commonly used; HOG88 slightly more stable
- **CB05:** Quadratic stable terms provide better performance for very stable conditions
- **Critical Ri_g ~0.25:** Above this, turbulence transitions to laminar flow
- **z/z₀ dependence:** Ri_b scales with roughness ratio (set to 1000 here)

### Next Steps

- Integrate into climate models (CESM, WRF, ICON)
- Develop ML-based bias correction using these tables
- Create pedagogical materials for ABL education